# Saving and Loading Models

When it comes to saving and loading models, there are 3 core functions:

- ***torch.save***: Saves a serialized object to disk using Python's **pickle** utility for serialization. Models, tensors, and dictionaries of all kinds of objects can be saved.

- ***torch.load***: Uses pickle's uppickling facilities to deserialize pickled object files to memory.

- ***torch.nn.Module.load_state_dict***: Loads a model's parameter dictionary using a deserialized state_dict.

## 1. What is state_dict?

In PyTorch, the learnable parameters (i.e. weights and biases) of an *torch.nn.Module* model are contained in the model's parameters (accessed with *model.parameters()*). A *state_dict* is simply a Python dictionary object that maps each layer to its parameter tensor.

Note that only layers with learnable parameters (convolutional layers, linear layers, etc.) and registered buffers have entries in the model's *state_dict*.

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
model = TheModelClass()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [5]:
print("Model's state_dict: ")
for param_tensor in model.state_dict():
    print(param_tensor, '\t', model.state_dict()[param_tensor].size())  # conv1,conv2,fc1,fc2,fc3的weight和bias

Model's state_dict: 
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])


In [6]:
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, '\t', optimizer.state_dict()[var_name])  # state和param_groups

Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [4753872288, 4753873296, 4753874736, 4753873152, 4753874160, 4753873368, 4753874376, 4753874520, 4753874232, 4753874880]}]


## 2. Saving & Loading Model for Inference

### Save/Load state_dict (Recommended)

***torch.save()*** function gives you the most flexibility for restoring the model later, which is why it's the recommended method for saving models.

A common PyTorch convention is to save models using either a ***.pt*** or ***.pth*** file extension.

Remember that you must call ***model.eval()*** to set **dropout and batch normalization layers** to evaluation mode before running inference. Failing to do this will yield **inconsistent inference result**.

#### Save:

```
torch.save(model.state_dict(), PATH)
```

#### Load:

```
model = TheModelClass(*args, **kwars)
model.load_state_dict(torch.load(PATH))
model.eval
```

### Save/Load Entire Model

Saving a model in this way will save the entire module using Python's *pickle* module.

The disadvantage of this approach is that the serialized data is **bound to the specific classes and the exact directory structure** used when the model is saved. The reason for this is bacause pickle doesn't save the model class itself. Rather, it saves a path to the file containing the class.

#### Save:

```
torch.save(model, PATH)
```

#### Loads:

```
model = torch.load(PATH)  # Model class must be defined somewhere
model.eval()
```

## 3. Saving & Loading a General Checkpoint for Inference and/or Resuming Training

When saving a general checkpoint, to be used for either inference or resuming training, you must save more than just **the model's state_dict**. It's important to alse save **the optimizer's state_dict**, as this contains buffers and parameters that are updated as the model trains. Other items that you may want to save are **epoch you left off on, the latest recorded training loss, external torch.nn.Embedding layer**, etc.

A common PyTorch convention is to save these checkpoints using the ***.tar*** file extension.

#### Save:

```
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss,
    ...
    }, PATH)
```

#### Load:

```
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - Or -
model.train()
```

## 4. Saving Multiple Models in One File

#### Save:

```
torch.save({
            'modelA_state_dict': modelA.state_dict(),
            'modelB_state_dict': modelB.state_dict(),
            'optimizerA_state_dict': optimizerA.state_dict(),
            'optimizerB_state_dict': optimizerB.state_dict(),
            ...
            }, PATH)
```

#### Load:

```
modelA = TheModelAClass(*args, **kwargs)
modelB = TheModelBClass(*args, **kwargs)
optimizerA = TheOptimizerAClass(*args, **kwargs)
optimizerB = TheOptimizerBClass(*args, **kwargs)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict'])
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval()
modelB.eval()
# - or -
modelA.train()
modelB.train()
```

## 5. Warmstarting Model Using Parameter Model

Partially loading a model or loading a partial model are common scenarios when transfer learning or training a new complex model.

Whether you are loading from ***a partial state_dict***, which is **missing some keys**, or loading a *state_dict* with **more keys than the model** that you are loading into, you can set the ***strict*** argument to ***False*** in the *load_state_dict()* function to ignore non-matching keys.

If you want to load parameters from one layer to another, but some keys don't match, simply **change the name of the parameter keys** in the *state_dict** that you are loading to match the keys in the model that you are loading into.

> 刘尧：可以**在不同结构和参数的model间互相load**! 只填充相同key的parameters，有时也可通过修改name来填充到指定的parameters！

#### Save:

```
torch.save(modelA.state_dict(), PATH)
```

#### Load:

```
modelB = TheModelBClass(*args, **kwargs)
modelB.load_state_dict(torch.load(PATH), strict=False)
```

## 6. Saving & Loading Model Across Devices

> 刘尧：总结为一句话就是：Load时，跨Devices需要map_location，在GPU上需要model.to(device)和tensor.ot(device)

#### Save:

```
torch.save(model.state_dict(), PATH)
```

### Save on GPU, Load on CPU

#### Load:

```
device = torch.device('cpu')
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location=device))
```

### Save on GPU, Load on GPU

#### Load:

```
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)
# Make sure to call input = input.to(device) on any input tensors that you feed to the model
```

### Save on CPU, Load on GPU

#### Load:

```
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))
model.to(device)
# Make sure to call input = input.to(device) on any input tensors that you feed to the model
```

### Saving torch.nn.DataParallel

***torch.nn.DataParallel*** is a model wrapper that enables parallel GPU utilization. To save a *DataParallel* model generically, save the ***model.module.state_dict()***. This way, you have the flexibility to load the model any way you want to any device you want.

#### Save:

```
torch.save(model.module.state_dict(), PATH)
```

#### Load:

```
# Load to whatever device you want
```